In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler


In [12]:
customers_df=pd.read_csv("C:/Users/Anamika/Downloads/Customers.csv")
transactions_df=pd.read_csv("C:/Users/Anamika/Downloads/Transactions.csv")

In [20]:
transactions_products = pd.merge(transactions, products, on='ProductID', how='left')
merged_data = pd.merge(transactions_products, customers, on='CustomerID', how='left')

In [22]:
customer_features = merged_data.groupby('CustomerID').agg({
    'Quantity': 'sum',          # Total quantity purchased
    'TotalValue': 'sum',        # Total transaction value            
    'Region': 'first',          # Customer's region
    'Category': lambda x: ','.join(x.unique())  # Purchased product categories
}).reset_index()

In [23]:
customer_features = pd.get_dummies(customer_features, columns=['Region'], drop_first=True)

In [24]:
for category in products['Category'].unique():
    customer_features[category] = customer_features['Category'].apply(lambda x: 1 if category in x else 0)
customer_features.drop(columns=['Category'], inplace=True)

In [25]:
scaler = MinMaxScaler()
numeric_cols = ['Quantity', 'TotalValue']
customer_features[numeric_cols] = scaler.fit_transform(customer_features[numeric_cols])

In [26]:
features = customer_features.drop(columns=['CustomerID']).values
customer_ids = customer_features['CustomerID'].tolist()

In [27]:
def get_similar_customers(customer_id, top_n=3):
    # Find the index of the customer in the dataset
    idx = customer_ids.index(customer_id)
    # Calculate cosine similarity with all other customers
    similarity_scores = cosine_similarity([features[idx]], features)[0]
    # Get top N most similar customers (exclude self)
    similar_customers = sorted(
        [(customer_ids[i], similarity_scores[i]) for i in range(len(similarity_scores)) if i != idx],
        key=lambda x: x[1], reverse=True
    )[:top_n]
    return similar_customers

In [28]:
customer_id = 'C0001'
top_3_recommendations = get_similar_customers(customer_id)
print(f"Top 3 similar customers for {customer_id}: {top_3_recommendations}")

Top 3 similar customers for C0001: [('C0174', 0.999777842455864), ('C0152', 0.9995148187663395), ('C0085', 0.9963755199349082)]


In [29]:
# Calculate similarity for the first 20 customers
lookalike_map = {}
for idx in range(20):  # For first 20 customers
    customer_id = customer_ids[idx]
    # Compute similarity scores
    similarity_scores = cosine_similarity([features[idx]], features)[0]
    # Get top 3 similar customers (excluding self)
    top_3 = sorted(
        [(customer_ids[i], round(similarity_scores[i], 2)) for i in range(len(similarity_scores)) if i != idx],
        key=lambda x: x[1], reverse=True
    )[:3]
    lookalike_map[customer_id] = top_3

In [30]:
lookalike_df = pd.DataFrame({
    'CustomerID': list(lookalike_map.keys()),
    'Lookalikes': [str(val) for val in lookalike_map.values()]
})

lookalike_df.to_csv('Lookalike.csv', index=False)

In [31]:
# Display the recommendations
print("Lookalike.csv created with recommendations for the first 20 customers.")
print(lookalike_df.head())

Lookalike.csv created with recommendations for the first 20 customers.
  CustomerID                                         Lookalikes
0      C0001   [('C0085', 1.0), ('C0152', 1.0), ('C0174', 1.0)]
1      C0002  [('C0159', 1.0), ('C0134', 0.99), ('C0106', 0....
2      C0003   [('C0031', 1.0), ('C0091', 1.0), ('C0129', 1.0)]
3      C0004  [('C0012', 1.0), ('C0148', 1.0), ('C0001', 0.98)]
4      C0005  [('C0007', 1.0), ('C0140', 1.0), ('C0069', 0.82)]


In [4]:
import csv

data=[
  CustomerID                                         Lookalikes
0      C0001   [('C0085', 1.0), ('C0152', 1.0), ('C0174', 1.0)],
1      C0002  [('C0159', 1.0), ('C0134', 0.99), ('C0106', 0....,
2      C0003   [('C0031', 1.0), ('C0091', 1.0), ('C0129', 1.0)],
3      C0004  [('C0012', 1.0), ('C0148', 1.0), ('C0001', 0.98)],
4      C0005  [('C0007', 1.0), ('C0140', 1.0), ('C0069', 0.82)]
                                                                                                  
# Specify the file name
file_name = "Anamika_PM_Lookalike.csv"

# Write data to the CSV file
with open(file_name, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"CSV file '{file_name}' created successfully.")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (60614460.py, line 4)